In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

In [ ]:
%matplotlib inline
import cv2
import numpy as np
from tqdm import tqdm
import networkx as nx
import matplotlib.pyplot as plt
import random

In [ ]:
Trans_cluster = np.zeros((888,859,915), dtype=int)
Cor_cluster = np.zeros((888,859,915),dtype=int)

cnt = 0

for img_num in tqdm(range(587,850)):
    input_file_current = "../../workspace/flower_graph/adjacency_matrix/Output/Spectral_sys/12_junk-del-label-inte-sp/ORA-{0:03d}_k-means-200_junk-del-label-inte_sp.png".format(img_num)
    #input_file_current = "Output/Spectral_sys/12_junk-del-label-inte-sp/ORA-{0:03d}_k-means-200_junk-del-label-inte_sp.png".format(img_num)
    img_c = cv2.imread(input_file_current)
    
    height,width,c = img_c.shape
    
    img_c_rmb = np.zeros((height,width))
    color_code_c = np.zeros((200,3))
    
    i = 1
    
    for y in range(height):
        for x in range(width):
            if((img_c[y,x,0]== img_c[y,x,1]) and (img_c[y,x,1] == img_c[y,x,2])):
                img_c[y,x] = (0,0,0)
            else:
                if(i==1):
                    color_code_c[i] = img_c[y,x]
                    Trans_cluster[img_num,y,x] = i + cnt
                    i+=1
                else:
                    flag = 0
                    j = 1

                    while(flag != 1 and j <= i):
                        if((color_code_c[j]==img_c[y,x]).all()):
                            flag = 1
                            Trans_cluster[img_num,y,x] = j + cnt
                        else:
                            j += 1

                    if(flag == 0):
                        color_code_c[i] = img_c[y,x]
                        Trans_cluster[img_num,y,x] = i+cnt
                        i += 1
    
    cnt += (i-1)

In [ ]:
for img_num in tqdm(range(96,800)):
    input_file_current = "../../workspace/flower_graph/adjacency_matrix/Output/Spectral_sys_Coronal/08_junk-del-label-inte-sp/ORA-Coronal-{0:03d}_k-means-200_junk-del-label-inte_sp.png".format(img_num)
    #input_file_current = "Output/Spectral_sys_Coronal/08_junk-del-label-inte-sp/ORA-Coronal-{0:03d}_k-means-200_junk-del-label-inte_sp.png".format(img_num)
    img_c = cv2.imread(input_file_current)
    
    depth,width,c = img_c.shape
    
    img_c_rmb = np.zeros((depth,width))
    color_code_c = np.zeros((200,3))

    i = 1
    
    for z in range(38,301):
        z_abs = 887-z
        for x in range(width):
            if((img_c[z,x,0]== img_c[z,x,1]) and (img_c[z,x,1] == img_c[z,x,2])):
                img_c[z,x] = (0,0,0)
            else:
                if(i==1):
                    color_code_c[i] = img_c[z,x]
                    Cor_cluster[z_abs,img_num,x] = i+cnt
                    i+=1
                else:
                    flag = 0
                    j = 1

                    while(flag != 1 and j <= i):
                        if((color_code_c[j]==img_c[z,x]).all()):
                            flag = 1
                            Cor_cluster[z_abs,img_num,x] = i+cnt
                        else:
                            j += 1

                    if(flag == 0):
                        color_code_c[i] = img_c[z,x]
                        Cor_cluster[z_abs,img_num,x] = i+cnt
                        i += 1
    
    cnt += (i-1)

### 横断面のみの統合

In [ ]:
new_cl_num = np.zeros(cnt+30)
for i in tqdm(range(cnt+30)):
    new_cl_num[i] = i

new_cluster = [[i] for i in range(20000)]

for z in tqdm(range(587,849)):
#for z in tqdm(range(587,589)):
    ch_num_u = np.zeros((cnt+30,cnt+30))

    st_flag = 0
    st_num = 0
    fin_num = 0
    
    for y in range(50,830):
        for x in range(50,880):
            if(Trans_cluster[z,y,x]>0):

                for y_n in range(y-1,y+2):
                    for x_n in range(x-1,x+2):
                        if(Trans_cluster[z+1,y_n,x_n]>0):
                            ch_num_u[int(Trans_cluster[z+1,y_n,x_n]),int(Trans_cluster[z,y,x])] += 1

    st_num = Trans_cluster[z,:,:].max()
    fin_num = Trans_cluster[z+1,:,:].max()
    
    for i in range(st_num,fin_num+1):
        ch_num_axis = ch_num_u[i,:]
        result = np.argmax(ch_num_axis)
        
        min_cl = 50000
        min_flag = 0

        if(result>0):
            if(min_cl > new_cl_num[i]):
                min_cl = new_cl_num[i]
                min_flag = 1
            if(min_cl > new_cl_num[result]):
                min_cl = new_cl_num[result]
                min_flag = 2

            if(min_flag == 1 and new_cl_num[i]!=new_cl_num[result]):
                new_cluster[int(new_cl_num[i])].extend(new_cluster[int(new_cl_num[result])])
                new_cluster[int(new_cl_num[result])] = []

                for i_n in new_cluster[int(new_cl_num[i])]:
                    new_cl_num[int(i_n)] = new_cl_num[i]

            elif(min_flag == 2 and new_cl_num[i]!=new_cl_num[result]):
                new_cluster[int(new_cl_num[result])].extend(new_cluster[int(new_cl_num[i])])
                new_cluster[int(new_cl_num[i])] = []

                for i_n in new_cluster[int(new_cl_num[result])]:
                    new_cl_num[int(i_n)] = new_cl_num[result]
            

### 横断面のみの統合（複数枚）

In [ ]:
new_cl_num = np.zeros(cnt+30)

for i in tqdm(range(cnt+30)):
    new_cl_num[i] = i

new_cluster = [[i] for i in range(20000)]

for z in tqdm(range(587,849)):
#for z in tqdm(range(587,589)):
    ch_num_u_1= np.zeros((cnt+30,cnt+30))
    ch_num_u_2= np.zeros((cnt+30,cnt+30))
    ch_num_u_3= np.zeros((cnt+30,cnt+30))
    ch_num_u_4= np.zeros((cnt+30,cnt+30))

    st_flag = 0
    st_num = 0
    fin_num = 0
    
    for y in range(50,830):
        for x in range(50,880):
            if(Trans_cluster[z,y,x]>0):

                for y_n in range(y-1,y+2):
                    for x_n in range(x-1,x+2):
                        if(Trans_cluster[z+1,y_n,x_n]>0):
                            ch_num_u_1[int(Trans_cluster[z+1,y_n,x_n]),int(Trans_cluster[z,y,x])] += 1
                        if(Trans_cluster[z+2,y_n,x_n]>0):
                            ch_num_u_2[int(Trans_cluster[z+2,y_n,x_n]),int(Trans_cluster[z,y,x])] += 1
                        """
                        if(Trans_cluster[z+3,y_n,x_n]>0):
                            ch_num_u_3[int(Trans_cluster[z+3,y_n,x_n]),int(Trans_cluster[z,y,x])] += 1
                        if(Trans_cluster[z+4,y_n,x_n]>0):
                            ch_num_u_4[int(Trans_cluster[z+4,y_n,x_n]),int(Trans_cluster[z,y,x])] += 1
                        """

    st_num = Trans_cluster[z,:,:].max()
    fin_num = Trans_cluster[z+1,:,:].max()
    
    for i in range(st_num,fin_num+1):
        if(i>0):
            ch_num_axis = ch_num_u_1[i,:]
            result = np.argmax(ch_num_axis)

            min_cl = 50000
            min_flag = 0

            if(result>0):
                if(min_cl > new_cl_num[i]):
                    min_cl = new_cl_num[i]
                    min_flag = 1
                if(min_cl > new_cl_num[result]):
                    min_cl = new_cl_num[result]
                    min_flag = 2

                if(min_flag ==1 and new_cl_num[i]!=new_cl_num[result]):
                    new_cluster[int(new_cl_num[i])].extend(new_cluster[int(new_cl_num[result])])
                    new_cluster[int(new_cl_num[result])] = []

                    for i_n in new_cluster[int(new_cl_num[i])]:
                        new_cl_num[int(i_n)] = new_cl_num[i]

                elif(min_flag == 2 and new_cl_num[i]!=new_cl_num[result]):
                    new_cluster[int(new_cl_num[result])].extend(new_cluster[int(new_cl_num[i])])
                    new_cluster[int(new_cl_num[i])] = []

                    for i_n in new_cluster[int(new_cl_num[result])]:
                        new_cl_num[int(i_n)] = new_cl_num[result]
    
    st_num = Trans_cluster[z+1,:,:].max()
    fin_num = Trans_cluster[z+2,:,:].max()
    
    for i in range(st_num,fin_num+1):
        if(i>0):
            ch_num_axis = ch_num_u_2[i,:]
            result = np.argmax(ch_num_axis)

            min_cl = 50000
            min_flag = 0

            if(result>0):
                if(min_cl > new_cl_num[i]):
                    min_cl = new_cl_num[i]
                    min_flag = 1
                if(min_cl > new_cl_num[result]):
                    min_cl = new_cl_num[result]
                    min_flag = 2

                if(min_flag == 1 and new_cl_num[i]!=new_cl_num[result]):
                    new_cluster[int(new_cl_num[i])].extend(new_cluster[int(new_cl_num[result])])
                    new_cluster[int(new_cl_num[result])] = []

                    for i_n in new_cluster[int(new_cl_num[i])]:
                        new_cl_num[int(i_n)] = new_cl_num[i]

                elif(min_flag == 2 and new_cl_num[i]!=new_cl_num[result]):
                    new_cluster[int(new_cl_num[result])].extend(new_cluster[int(new_cl_num[i])])
                    new_cluster[int(new_cl_num[i])] = []

                    for i_n in new_cluster[int(new_cl_num[result])]:
                        new_cl_num[int(i_n)] = new_cl_num[result]
    
    """
    st_num = Trans_cluster[z+2,:,:].max()
    fin_num = Trans_cluster[z+3,:,:].max()
    
    for i in range(st_num,fin_num+1):
        if(i>0):
            ch_num_axis = ch_num_u_3[i,:]
            result = np.argmax(ch_num_axis)

            min_cl = 50000
            min_flag = 0

            if(result>0):
                if(min_cl > new_cl_num[i]):
                    min_cl = new_cl_num[i]
                    min_flag = 1
                if(min_cl > new_cl_num[result]):
                    min_cl = new_cl_num[result]
                    min_flag = 2

                if(min_flag == 1 and new_cl_num[i]!=new_cl_num[result]):
                    new_cluster[int(new_cl_num[i])].extend(new_cluster[int(new_cl_num[result])])
                    new_cluster[int(new_cl_num[result])] = []

                    for i_n in new_cluster[int(new_cl_num[i])]:
                        new_cl_num[int(i_n)] = new_cl_num[i]

                elif(min_flag == 2 and new_cl_num[i]!=new_cl_num[result]):
                    new_cluster[int(new_cl_num[result])].extend(new_cluster[int(new_cl_num[i])])
                    new_cluster[int(new_cl_num[i])] = []

                    for i_n in new_cluster[int(new_cl_num[result])]:
                        new_cl_num[int(i_n)] = new_cl_num[result]
                    
    st_num = Trans_cluster[z+3,:,:].max()
    fin_num = Trans_cluster[z+4,:,:].max()
    
    for i in range(st_num,fin_num+1):
        if(i>0):
            ch_num_axis = ch_num_u_4[i,:]
            result = np.argmax(ch_num_axis)

            min_cl = 50000
            min_flag = 0

            if(result>0):
                if(min_cl > new_cl_num[i]):
                    min_cl = new_cl_num[i]
                    min_flag = 1
                if(min_cl > new_cl_num[result]):
                    min_cl = new_cl_num[result]
                    min_flag = 2

                if(min_flag == 1 and new_cl_num[i]!=new_cl_num[result]):
                    new_cluster[int(new_cl_num[i])].extend(new_cluster[int(new_cl_num[result])])
                    new_cluster[int(new_cl_num[result])] = []

                    for i_n in new_cluster[int(new_cl_num[i])]:
                        new_cl_num[int(i_n)] = new_cl_num[i]

                elif(min_flag == 2 and new_cl_num[i]!=new_cl_num[result]):
                    new_cluster[int(new_cl_num[result])].extend(new_cluster[int(new_cl_num[i])])
                    new_cluster[int(new_cl_num[i])] = []

                    for i_n in new_cluster[int(new_cl_num[result])]:
                        new_cl_num[int(i_n)] = new_cl_num[result]
    """

In [ ]:
for z in tqdm(range(849,590,-1)):
#for z in tqdm(range(587,589)):
    ch_num_u_1= np.zeros((cnt+30,cnt+30))

    st_flag = 0
    st_num = 0
    fin_num = 0
    
    for y in range(50,830):
        for x in range(50,880):
            if(Trans_cluster[z,y,x]>0):

                for y_n in range(y-1,y+2):
                    for x_n in range(x-1,x+2):
                        if(Trans_cluster[z-1,y_n,x_n]>0):
                            ch_num_u_1[int(Trans_cluster[z-1,y_n,x_n]),int(Trans_cluster[z,y,x])] += 1

    st_num = Trans_cluster[z-2,:,:].max()
    fin_num = Trans_cluster[z-1,:,:].max()
    
    for i in range(st_num,fin_num+1):
        ch_num_axis = ch_num_u_1[i,:]
        result = np.argmax(ch_num_axis)
        
        min_cl = 50000
        min_flag = 0

        if(result>0):
            if(min_cl > new_cl_num[i]):
                min_cl = new_cl_num[i]
                min_flag = 1
            if(min_cl > new_cl_num[result]):
                min_cl = new_cl_num[result]
                min_flag = 2

            if(min_flag == 1 and new_cl_num[i]!=new_cl_num[result]):
                new_cluster[int(new_cl_num[i])].extend(new_cluster[int(new_cl_num[result])])
                new_cluster[int(new_cl_num[result])] = []

                for i_n in new_cluster[int(new_cl_num[i])]:
                    new_cl_num[int(i_n)] = new_cl_num[i]

            elif(min_flag == 2 and new_cl_num[i]!=new_cl_num[result]):
                new_cluster[int(new_cl_num[result])].extend(new_cluster[int(new_cl_num[i])])
                new_cluster[int(new_cl_num[i])] = []

                for i_n in new_cluster[int(new_cl_num[result])]:
                    new_cl_num[int(i_n)] = new_cl_num[result]

### 冠状面のみの統合

In [ ]:
new_cl_num = np.zeros(cnt+30)
for i in tqdm(range(cnt+30)):
    new_cl_num[i] = i

new_cluster = [[i] for i in range(20000)]

for y in tqdm(range(96,800)):
#for z in tqdm(range(587,589)):
    ch_num_u = np.zeros((cnt+30,cnt+30))

    st_flag = 0
    st_num = 0
    fin_num = 0
    
    for z in range(587,849):
        for x in range(50,880):
            if(Cor_cluster[z,y,x]>0):
                
                for z_n in range(z-1,z+2):
                    for x_n in range(x-1,x+2):
                        if(Trans_cluster[z_n,y+1,x_n]>0):
                            ch_num_u[int(Cor_cluster[z_n,y+1,x_n]),int(Cor_cluster[z,y,x])] += 1

    st_num = Cor_cluster[:,y,:].max()
    fin_num = Cor_cluster[:,y+1,:].max()

    for i in range(st_num,fin_num+1):
        ch_num_axis = ch_num_u[i,:]
        result = np.argmax(ch_num_axis)
        
        min_cl = 50000
        min_flag = 0

        if(result>0):
            if(min_cl > new_cl_num[i]):
                min_cl = new_cl_num[i]
                min_flag = 1
            if(min_cl > new_cl_num[result]):
                min_cl = new_cl_num[result]
                min_flag = 2

            if(min_flag == 1 and new_cl_num[i]!=new_cl_num[result]):
                new_cluster[int(new_cl_num[i])].extend(new_cluster[int(new_cl_num[result])])
                new_cluster[int(new_cl_num[result])] = []

                for i_n in new_cluster[int(new_cl_num[i])]:
                    new_cl_num[int(i_n)] = new_cl_num[i]

            elif(min_flag == 2 and new_cl_num[i]!=new_cl_num[result]):
                new_cluster[int(new_cl_num[result])].extend(new_cluster[int(new_cl_num[i])])
                new_cluster[int(new_cl_num[i])] = []

                for i_n in new_cluster[int(new_cl_num[result])]:
                    new_cl_num[int(i_n)] = new_cl_num[result]
            

### 横断面と冠状面の両方

In [ ]:
new_cl_num = np.zeros(cnt+30)
for i in tqdm(range(cnt+30)):
    new_cl_num[i] = i

new_cluster = [[i] for i in range(20000)]

for z in tqdm(range(587,849)):
#for z in tqdm(range(587,589)):
    ch_num_u = np.zeros((cnt+30,cnt+30))

    st_flag = 0
    st_num = 0
    fin_num = 0
    
    for y in range(50,830):
        for x in range(50,880):
            if(Trans_cluster[z,y,x]>0):

                for y_n in range(y-1,y+2):
                    for x_n in range(x-1,x+2):
                        if(Trans_cluster[z+1,y_n,x_n]>0):
                            ch_num_u[int(Trans_cluster[z+1,y_n,x_n]),int(Trans_cluster[z,y,x])] += 1
                            
            if(Cor_cluster[z,y,x]>0):
                i = int(Trans_cluster[z,y,x])
                result = int(Cor_cluster[z,y,x])
                if(new_cl_num[i]!=new_cl_num[result]):
                    new_cluster[int(new_cl_num[i])].extend(new_cluster[int(new_cl_num[result])])
                    new_cluster[int(new_cl_num[result])] = []

    st_num = Trans_cluster[z,:,:].max()
    fin_num = Trans_cluster[z+1,:,:].max()
    
    for i in range(st_num,fin_num+1):
        ch_num_axis = ch_num_u[i,:]
        result = np.argmax(ch_num_axis)
        
        min_cl = 50000
        min_flag = 0

        if(result>0):
            if(min_cl > new_cl_num[i]):
                min_cl = new_cl_num[i]
                min_flag = 1
            if(min_cl > new_cl_num[result]):
                min_cl = new_cl_num[result]
                min_flag = 2

            if(min_flag == 1 and new_cl_num[i]!=new_cl_num[result]):
                new_cluster[int(new_cl_num[i])].extend(new_cluster[int(new_cl_num[result])])
                new_cluster[int(new_cl_num[result])] = []

                for i_n in new_cluster[int(new_cl_num[i])]:
                    new_cl_num[int(i_n)] = new_cl_num[i]

            elif(min_flag == 2 and new_cl_num[i]!=new_cl_num[result]):
                new_cluster[int(new_cl_num[result])].extend(new_cluster[int(new_cl_num[i])])
                new_cluster[int(new_cl_num[i])] = []

                for i_n in new_cluster[int(new_cl_num[result])]:
                    new_cl_num[int(i_n)] = new_cl_num[result]
            

In [ ]:
for z in tqdm(range(849,587,-1)):

    st_flag = 0
    st_num = 0
    fin_num = 0
    
    for y in range(50,830):
        for x in range(50,880):
            if(Trans_cluster[z,y,x]>0):
                if(st_flag==0):
                    st_flag = 1
                    st_num = Trans_cluster[z,y,x]
                for y_n in range(y-1,y+2):
                    for x_n in range(x-1,x+2):
                        if(Trans_cluster[z-1,y_n,x_n]>0):
                            ch_num_u[int(Trans_cluster[z,y,x]),int(Trans_cluster[z-1,y_n,x_n])] += 1

    fin_num = Trans_cluster[z,:,:].max()
    
    for i in range(st_num,fin_num+1):
        ch_num_axis = ch_num_u[i,:]
        result = np.argmax(ch_num_axis)
        
        min_cl = 50000
        min_flag = 0

        if(result>0):
            if(min_cl > new_cl_num[i]):
                min_cl = new_cl_num[i]
                min_flag = 1
            if(min_cl > new_cl_num[result]):
                min_cl = new_cl_num[result]
                min_flag = 2

            if(min_flag == 1):
                new_cluster[i].extend(new_cluster[result])
                new_cluster[result] = []

                for i_n in new_cluster[i]:
                    new_cl_num[int(i_n)] = new_cl_num[i]

            elif(min_flag == 2):
                new_cluster[result].extend(new_cluster[i])
                new_cluster[i] = []

                for i_n in new_cluster[result]:
                    new_cl_num[int(i_n)] = new_cl_num[result]

### 配列segに統合結果を格納

In [ ]:
seg = np.zeros((888,859,915), dtype=int)

for z in tqdm(range(587,849)):
#for z in tqdm(range(645,651)):
    for y in range(50,830):
        for x in range(50,880):
            """
            if(Trans_cluster[z,y,x]>0 and Cor_cluster[z,y,x]>0):
                seg[z,y,x] = new_cl_num[int(Trans_cluster[z,y,x])]
            elif(Trans_cluster[z,y,x]>0):
                seg[z,y,x] = new_cl_num[int(Trans_cluster[z,y,x])]
            elif(Cor_cluster[z,y,x]>0):
                seg[z,y,x] = new_cl_num[int(Cor_cluster[z,y,x])]
            """
            
            if(Trans_cluster[z,y,x]>0):
                seg[z,y,x] = new_cl_num[int(Trans_cluster[z,y,x])]
            """
            if(Cor_cluster[z,y,x]>0):
                seg[z,y,x] = new_cl_num[int(Cor_cluster[z,y,x])]
            """

### 統合後の番号に合わせて色付け

In [ ]:
colors = []
for i in range(1, 30000):
    colors.append(np.array([random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]))
    
#for img_num in tqdm(range(645,651)):
for img_num in tqdm(range(587,849)):
    input_file = "Output/Spectral_sys/12_junk-del-label-inte-sp/ORA-{0:03d}_k-means-200_junk-del-label-inte_sp.png".format(img_num)
    output_file = "Output/3D_inte/3D-inte_graph_Trans_reverse/ORA-{0:03d}_3D-inte_graph_Trans_reverse.png".format(img_num)

    #input_file = "../../workspace/flower_graph/adjacency_matrix/Input/ORA-PNG/ORA-{0:03d}.png".format(img_num)
    #output_file = "Output/3D-inte_ORA-{0:03d}.png".format(img_num)
    
    img_c = cv2.imread(input_file)
    
    height,width,c = img_c.shape
    for y in range(height):
        for x in range(width):
            if(seg[img_num,y,x]>0):
                img_c[y,x] = colors[int(seg[img_num,y,x])]
            else:
                img_c[y,x] = (0,0,0)
                
    cv2.imwrite(output_file,img_c)

In [ ]:
colors = []
for i in range(1, 30000):
    colors.append(np.array([random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]))
    
#for img_num in tqdm(range(645,651)):
for img_num in tqdm(range(587,849)):
    input_file = "Output/Spectral_sys/12_junk-del-label-inte-sp/ORA-{0:03d}_k-means-200_junk-del-label-inte_sp.png".format(img_num)
    output_file = "Output/3D_inte/3D-inte_graph_CorTrans/ORA-{0:03d}_3D-inte_graph_CorTrans.png".format(img_num)

    #input_file = "../../workspace/flower_graph/adjacency_matrix/Input/ORA-PNG/ORA-{0:03d}.png".format(img_num)
    #output_file = "Output/3D-inte_ORA-{0:03d}.png".format(img_num)
    
    img_c = cv2.imread(input_file)
    
    height,width,c = img_c.shape
    for y in range(height):
        for x in range(width):
            if(seg[img_num,y,x]>0):
                img_c[y,x] = colors[int(seg[img_num,y,x])]
            else:
                img_c[y,x] = (0,0,0)
                
    cv2.imwrite(output_file,img_c)

In [ ]:
colors = []
for i in range(1, 30000):
    colors.append(np.array([random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]))
    
#for img_num in tqdm(range(645,651)):
for img_num in tqdm(range(587,849)):
    input_file = "Output/Spectral_sys/12_junk-del-label-inte-sp/ORA-{0:03d}_k-means-200_junk-del-label-inte_sp.png".format(img_num)
    output_file = "Output/3D_inte/3D-inte_graph_Cor/ORA-{0:03d}_3D-inte_graph_Cor.png".format(img_num)

    #input_file = "../../workspace/flower_graph/adjacency_matrix/Input/ORA-PNG/ORA-{0:03d}.png".format(img_num)
    #output_file = "Output/3D-inte_ORA-{0:03d}.png".format(img_num)
    
    img_c = cv2.imread(input_file)
    
    height,width,c = img_c.shape
    for y in range(height):
        for x in range(width):
            if(seg[img_num,y,x]>0):
                img_c[y,x] = colors[int(seg[img_num,y,x])]
            else:
                img_c[y,x] = (0,0,0)
                
    cv2.imwrite(output_file,img_c)

### networkx

In [ ]:
G = nx.Graph()

for i in range(1,cnt+1):
    G.add_node(i)
    
check = np.zeros((cnt+30,cnt+30))

for z in tqdm(range(850-587-1)):
    for y in range(50,830):
        for x in range(50,880):
            if(Trans_cluster[z,y,x]>0):
                if(Trans_cluster[z+1,y,x]>0 and check[int(Trans_cluster[z,y,x]),int(Trans_cluster[z+1,y,x])]==0):
                    G.add_edge(int(Trans_cluster[z,y,x]),int(Trans_cluster[z+1,y,x]))
                if(Cor_cluster[z,y,x]>0 and check[int(Trans_cluster[z,y,x]),int(Cor_cluster[z,y,x])]==0):
                    G.add_edge(int(Trans_cluster[z,y,x]),int(Cor_cluster[z,y,x]))
                if(Cor_cluster[z+1,y,x]>0 and check[int(Trans_cluster[z,y,x]),int(Cor_cluster[z+1,y,x])]==0):
                    G.add_edge(int(Trans_cluster[z,y,x]),int(Cor_cluster[z+1,y,x]))

In [ ]:
#pos = nx.spring_layout(G)

plt.figure(figsize=(15,15))
nx.draw_networkx(G,with_labels=False, node_shape='.')
plt.axis("off")

In [ ]:
test = Trans_cluster[587,:,:]
img_NLMD = np.where(test==10 ,255,0)
cv2.imwrite('test_inte.png',img_NLMD)

In [ ]:
fig = plt.figure()
plt.figure(figsize=(30, 30))

#表示
plt.imshow(test,cmap='gray')